In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, RepeatVector, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load ABM dataset
file_path = '/Users/mac/Desktop/abm.csv'
data = pd.read_csv(file_path)

# =================== Step 1: Data Preprocessing ===================
# Fill missing values
data['country'].fillna('Unknown', inplace=True)
data['province'].fillna('Unknown', inplace=True)
data['city'].fillna('Unknown', inplace=True)

# Convert time and date to a single datetime column
data['datetime'] = pd.to_datetime(data['transaction_date'] + ' ' + data['transaction_time'])
data = data.sort_values(by='datetime')

# Normalize numeric columns
scaler = MinMaxScaler()
data['amount_cad'] = scaler.fit_transform(data[['amount_cad']])

# Encode categorical variables (e.g., debit_credit, cash_indicator)
data = pd.get_dummies(data, columns=['debit_credit', 'cash_indicator'])

# Dynamically identify dummy-encoded column names
encoded_columns = [col for col in data.columns if col.startswith('debit_credit') or col.startswith('cash_indicator')]

# Sequence preparation (group transactions by customer_id and create time-series data)
sequence_data = []
customer_ids = data['customer_id'].unique()

for customer_id in customer_ids:
    customer_data = data[data['customer_id'] == customer_id]
    sequence = customer_data[['amount_cad'] + encoded_columns].values
    if len(sequence) > 10:  # Only include sequences longer than 10
        sequence_data.append(sequence)

# Pad sequences to the same length
max_seq_len = max(len(seq) for seq in sequence_data)
sequence_data = tf.keras.preprocessing.sequence.pad_sequences(sequence_data, maxlen=max_seq_len, dtype='float32')

# Split data into training and testing sets
X_train, X_test = train_test_split(sequence_data, test_size=0.2, random_state=42)

# =================== Step 2: Bidirectional LSTM Model Architecture ===================
model = Sequential([
    Bidirectional(LSTM(64, activation='relu', return_sequences=True), input_shape=(max_seq_len, X_train.shape[2])),
    Bidirectional(LSTM(32, activation='relu', return_sequences=False)),
    RepeatVector(max_seq_len),
    Bidirectional(LSTM(32, activation='relu', return_sequences=True)),
    Bidirectional(LSTM(64, activation='relu', return_sequences=True)),
    TimeDistributed(Dense(X_train.shape[2]))
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# =================== Step 3: Training ===================
# Train the Bidirectional LSTM autoencoder on normal sequences (no anomalies)
history = model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1, shuffle=True)

# =================== Step 4: Anomaly Detection ===================
# Compute reconstruction error on test data
def detect_anomalies(model, data, threshold=None):
    reconstructions = model.predict(data)
    reconstruction_errors = np.mean(np.square(reconstructions - data), axis=(1, 2))
    
    if threshold is None:
        threshold = np.percentile(reconstruction_errors, 95)  # Set threshold at the 95th percentile

    anomalies = reconstruction_errors > threshold
    return anomalies, reconstruction_errors, threshold

# Detect anomalies
anomalies, errors, threshold = detect_anomalies(model, X_test)

# Output anomaly statistics
print(f"Threshold for anomaly detection: {threshold}")
print(f"Number of anomalies detected: {np.sum(anomalies)} out of {len(X_test)}")


/Users/mac/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/r7/ks7k8hn17dqdqv3766yjwxph0000gn/T/ipykernel_14610/3455276958.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['country'].fillna('Unknown', inplace=True)
/var/folders/r7/ks7k8hn17dqdqv3766yjwxph0000gn/T/ipykernel_14610/3455276958.py:16: Fu

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 256, 128)       │        35,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 256, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 256, 64)        │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 256, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 256, 5)         │           645 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,581 (658.52 KB)

 Trainable params: 168,581 (658.52 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 0.0334 - val_loss: 0.0339
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 0.0307 - val_loss: 0.0331
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 972ms/step - loss: nan - val_loss: nan
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 899ms/step - loss: nan - val_loss: nan
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: nan - val_loss: nan
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - loss: nan - val_loss: nan
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - loss: nan - val_loss: nan
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: nan - val_loss: nan
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: nan - val_loss: nan
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: nan - val_loss: nan
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: nan - val_loss: nan
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: nan - val_loss: nan
Epoch 13/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 